<a href="https://colab.research.google.com/github/rishitarya/Resumebot/blob/main/Ask_Rishit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ask Rishit**
### This is a chatbot where you can ask professional questions to Rishit
#### *Press **Cmd/Cntrl + F9** to run this notebook, you will see the chatbot loaded below withing 4 mins (Connect to T4 runtime if not by default)*

######This chatbot is based on fine tuned phi3 model and gte embeddings.
######Please feel free to reach out to me at [Linkedin](https://www.linkedin.com/in/rishit-arya/)

In [ ]:
#@title Loading Libraries
%%capture
!pip install -q gradio;
!pip install -q langchain;
!pip install -q langchain_community;
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install ".[xformers]" --upgrade --force-reinstall --extra-index-url https://download.pytorch.org/whl/cu118;
#!pip install -q --no-deps "xformers<0.0.26" "trl<0.9.0" peft accelerate bitsandbytes
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install torch==2.0.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install bitsandbytes
!pip install -q sentence_transformers;
!pip install -q huggingface_hub;
!pip install -q faiss-gpu;
!pip install -q faiss;
!pip install -q triton triton.common;

In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
#@title Loading the model
%%capture
from unsloth import FastLanguageModel
from huggingface_hub import hf_hub_download
from langchain.embeddings import GooglePalmEmbeddings, HuggingFaceEmbeddings
import faiss.contrib.torch_utils
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import torch

model_name = 'aryarishit/mistral-unsloth-resumebot-lora_adaptar-v2'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
)


FastLanguageModel.for_inference(model)

with open(hf_hub_download(repo_id="aryarishit/Info_docs", filename="Info_doc.txt",repo_type='dataset')) as f:
    contexts = f.read()
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=256,
    separators = ['\n\n','\n','  '],
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)
contexts = text_splitter.split_text(contexts)
contexts = [x.strip() for x in contexts]
contexts = np.array([x for x in contexts if len(x) > 0])

encoder = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5',trust_remote_code=True)
context_vectors = encoder.encode(contexts)
print(1)
print(context_vectors)
index = faiss.IndexFlatL2(context_vectors.shape[1])
faiss.normalize_L2(context_vectors)
index.add(context_vectors)

def get_context(query,top_k = 3):
    top_k = top_k

    query_vector = encoder.encode([query])
    faiss.normalize_L2(query_vector)
    top_k = top_k
    distances, indices = index.search(query_vector, top_k)
    context = ""
    for i in range(len(indices[0])):
        context = context + contexts[indices[0][i]] + "\n\n"
    return context

def get_alpaca_prompt(context,query):
    instruction_string = '''[INST]Consider you are assistant to Rishit Arya, and answers on behalf of him, Given the following context and a question, generate an answer based on the given context only. If the answer to the question is not found in the context, strictly state "I was not provided this info" only, don't try to make up an answer.Answer pricesly to what is asked it as if you are answering to Rishit's potential client. \nContext:{}
Question:{}[\INST] \nAnswer:'''

    prompt = instruction_string.format(
            context,
            query # input
        )
    return prompt

def generate_response(query,history):
    context = get_context( query)
    print(context)
    prompt = get_alpaca_prompt(context,query)
    # print(prompt)
    inputs = tokenizer([prompt],return_tensors='pt').to('cuda')
    outputs = model.generate(**inputs, max_new_tokens = 200,use_cache=True)
    return tokenizer.batch_decode(outputs)[0].split('Answer:')[1].replace('<|endoftext|>','')

In [ ]:
#@title Loading the app
import gradio as gr

gr.ChatInterface(generate_response,
                 title='Ask Rishit',
                 chatbot=gr.Chatbot(value=[(None, "Welcome 👋. What would you like to know?")],height=250),
                 examples = ['Tell me about his medium articles','What are his previous experiences?','What is his educational background?']).launch(debug=True)

<ipython-input-5-6586eb4a12d5>:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(value=[(None, "Welcome 👋. What would you like to know?")],height=250),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://38a117ec65c54acda3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Hello, My name is Rishit Arya. I am working as a Data Scientist at Walmart Global Tech India which is based in Bengaluru, India. I have a degree in Chamical Engineering from IIT BHU Varanasi. I have interest in Machine learning modelling and statistical analysis. I believe in power of data and the effect it can have in shaping the world, I am always looking for opportunities where I can explore and experience this power first hand.

I have two personal projects here, American type option pricing through modified Black Scholes model and Resume chatbot.

I love to play cricket, piano, researching about random stuffs, Here is the to my medium blogs where I try to write some of these researches as blog.




Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2103, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1648, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/utils.py", line 857, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://06a9587e0b66a3e34e.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://38a117ec65c54acda3.gradio.live
